In [18]:
import pandas as pd
import plotly.graph_objects as go

In [19]:
oneKB_strs = pd.read_csv("1000B_strings.csv")
oneKB_strs_tempfs = pd.read_csv("1000B_strings_tempfs.csv")

In [20]:
oneKB_strs

,keys,value_size,num_threads,rdbcompression,rdbchecksum,valkey_data_throughput_mb_s,actual_throughput_mb_s,rdb_file_size_bytes,status,port,save_duration_seconds,cpu_utilization_percent,cpu_total_time_seconds,io_read_bytes,io_write_bytes,memory_rss_bytes,context_switches_voluntary,context_switches_involuntary,iowait_time_seconds,iowait_percentage
0,10000000,1000,1,yes,yes,0,120.261253,10130117163,ok,7000,84.234256,101.656979,85.63,0,10130128896,11162882048,36,299,0.03,0.035615
1,10000000,1000,2,yes,yes,0,271.660810,10130339228,ok,7000,37.290396,197.852551,73.78,0,10130345984,11165560832,1784,78,0.03,0.080450
2,10000000,1000,3,yes,yes,0,344.624301,10130339251,ok,7000,29.395313,295.965549,87.00,0,10130341888,11166756864,694,78,0.04,0.136076
3,10000000,1000,4,yes,yes,0,473.622354,10130339272,ok,7000,21.389065,375.986516,80.42,0,10130345984,11167985664,1929,49,0.03,0.140259
4,10000000,1000,5,yes,yes,0,630.010376,10130339297,ok,7000,16.079639,494.040948,79.44,0,10130341888,11169239040,1326,40,0.08,0.497524
5,10000000,1000,6,yes,yes,0,755.044649,10130339272,ok,7000,13.416874,571.519099,76.68,0,10130341888,11170512896,2664,580,0.47,3.503051
6,10000000,1000,7,yes,yes,0,753.665963,10130339274,ok,7000,13.441418,556.860889,74.85,0,10130345984,11164057600,7518,775,2.41,17.929656
7,10000000,1000,8,yes,yes,0,755.011887,10130339320,ok,7000,13.417457,556.513813,74.67,0,10130341888,11165319168,6480,740,2.37,17.663556
8,10000000,1000,9,yes,yes,0,754.786869,10130339320,ok,7000,13.421457,559.402764,75.08,0,10130341888,11166621696,4587,253,2.21,16.466171
9,10000000,1000,10,yes,yes,0,755.245874,10130339341,ok,7000,13.413300,550.871158,73.89,0,10130345984,11165622272,3919,591,2.25,16.774396


In [29]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px # For distinct colors and markers

def plot_performance_by_workload_on_threads(dfs: list[pd.DataFrame], workload_names: list[str], title: str):
    """
    Generates an interactive Plotly graph showing speedup and efficiency
    vs. number of threads, with each line representing a different workload.

    Args:
        dfs (list[pd.DataFrame]): A list of DataFrames, where each DataFrame
                                   contains 'num_threads', 'value_size', 'speedup',
                                   and 'efficiency' columns for a specific workload.
                                   It's assumed 'speedup' and 'efficiency' have
                                   already been calculated for each df.
        workload_names (list[str]): A list of descriptive names for each workload,
                                     used for legend labels (e.g., ["50 Bytes", "500 Bytes"]).
                                     Length must match dfs.
    """
    if not dfs or not workload_names or len(dfs) != len(workload_names):
        print("Error: Input lists 'dfs' and 'workload_names' must not be empty and must have matching lengths.")
        return

    # Define fixed y-axis ranges for consistent comparison across all your plots
    # Adjust these ranges based on the MIN and MAX values you expect across ALL your datasets.
    maxes = [max(df["actual_throughput_mb_s"]) for df in dfs]
    max_throughput = max(maxes)
    speedup_y_range = [0.0, max_throughput + 250]  # Example: from 0x to 5x speedup

    # Get all unique thread counts across all DFs for x-axis ticks
    all_num_threads = sorted(pd.concat(df['num_threads'] for df in dfs).unique())

    # --- Color and Marker Definition ---
    colors = px.colors.qualitative.Plotly # A good default palette
    markers = ['circle', 'square', 'diamond', 'cross', 'x', 'triangle-up', 'triangle-down', 'pentagon', 'hexagram']
    # --- End Color and Marker Definition ---

    # Create subplots: one for Speedup, one for Efficiency
    fig = make_subplots(rows=1, cols=1,
                        subplot_titles=())

    # Plot Speedup for each workload
    for i, df in enumerate(dfs):
        workload_label = workload_names[i]
        color = colors[i % len(colors)]
        marker = markers[i % len(markers)]

        fig.add_trace(
            go.Scatter(
                x=df['num_threads'],
                y=df['actual_throughput_mb_s'],
                mode='lines+markers',
                name=f'Throughput: {workload_label}',
                marker=dict(symbol=marker, size=8),
                line=dict(width=2, color=color),
                legendgroup=workload_label, # Group traces for consistent legend
                showlegend=True
            ),
            row=1, col=1
        )

    # Update layout for a beautiful graph
    fig.update_layout(
        title_text=title,
        height=600, width=1200,
        showlegend=True,
        hovermode="x unified",
        template="plotly_white",
        title_x=0.5,
    )

    # Update x-axes for clarity
    fig.update_xaxes(
        title_text='Number of Threads',
        tickmode='array',
        tickvals=all_num_threads,
        row=1, col=1
    )

    # Update y-axes for clarity and consistent ranges
    fig.update_yaxes(title_text='Throughput (MB/s)', range=speedup_y_range, row=1, col=1)

    fig.show()
    # fig.write_html(output_filename)
    # You can also save it as an HTML file for interactive viewing later:
    # fig.write_html(f"performance_scaling_across_workloads.html")

In [30]:
plot_performance_by_workload_on_threads([oneKB_strs, oneKB_strs_tempfs], ["Saving to SSD File System", "Saving to in Memory File System"], "Save Throughput (MB/s) Vs. Num Threads when saving to different file systems")

In [31]:
load_df = pd.read_csv("load_summary_10.0keys_100B_comp-yes_csum-yes.csv")
plot_performance_by_workload_on_threads([load_df], ["Loading from SSD"], "Load Throughput (MB/s) Vs. Num Threads")